# Melting Dynamics

This is a notebook to understand the dynamics of crystal melting, in particular how much of the slowdown of the melting can be attributed to the slow dynamics.

In [1]:
# Import required modules
import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact, ToggleButtons, IntSlider

## Input data

The data for this notebook is sourced from the calculations of the relaxation times which can be run using the makefile in the home directory of this repository. The command `make relaxations` will compute the relaxation values that are required for this analysis, although if not run previously the command `make dynamics` will also need to be run as I am yet to set up any dependencies in the Makefile.

The calculation of the melting rates is performed in the [Crystal_Clustering](Crystal_Clustering.ipynb) notebook from calculations of crystal fractions from running `make melting`.

In [2]:
df_relax = pd.read_hdf('../data/analysis/dynamics.h5', 'relaxations')
df_relax.reset_index(inplace=True)
df_relax['1/D'] = 1 / df.diffusion_constant
df_relax['temperature'] = df_relax['temperature'].astype(float)
df_relax['pressure'] = df_relax['pressure'].astype(float)

df_melting = pd.read_hdf("../data/analysis/melting.h5", "rates")
df_melting['temperature'] = df_melting['temperature'].astype(float)
df_melting['pressure'] = df_melting['pressure'].astype(float)
df_melting['inv_melting_rate'] = np.abs(1/df_melting['melting_rate'].values)
df_melting.set_index(['temperature', 'pressure'], inplace=True)

NameError: name 'df' is not defined

## Merge Dataframes

This merges the dataframes containing the data on the timescale of relaxations and the timescale of melting.

In [ ]:
df_timescales = df_relax.merge(df_melting, on=['temperature', 'pressure'])
df_timescales['pressure'] = df_timescales['pressure'].astype(str)

In [ ]:
df_timescales.columns

In [ ]:
df_timescales.pressure.astype('str')

In [ ]:
def create_chart(axis):
    chart = alt.Chart(df_timescales, width=400, height=500).mark_point(size=75).transform_calculate(
        '1/T', '1/datum.temperature'
    ) .encode(
        x=alt.X('1/T:Q', scale=alt.Scale(zero=False)),
        y=alt.Y(axis, type='quantitative', axis=alt.Axis(format='e'), scale=alt.Scale(type='log')),
        color="crys:N",
        tooltip=[alt.Tooltip(f'{axis}:Q', format='.2e')]
    )
    return chart.transform_filter(alt.datum.pressure == '1.0')

In [ ]:
create_chart('inv_melting_rate') + create_chart('tau_T4_mean').mark_point(filled=True)

In [ ]:
df_timescales['tau_T4_melting_rate'] = df_timescales["tau_T4_mean"]*df_timescales["melting_rate"].abs()
df_timescales['timescale_per_event'] = 1/df_timescales['tau_T4_melting_rate']

In [ ]:
c = create_chart("tau_T4_melting_rate")

In [ ]:
c.mark_point(filled=False)

In [ ]:
df_timescales.query('pressure == "1.0"').melting_rate